In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# required packages 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Functions to  display summary of the dataset:


In [ ]:
def explore_numerical_types(df):
    # SUMMARY
    df_types = pd.DataFrame(df.dtypes, columns=['Data Type'])
    numerical_cols = df_types[~df_types['Data Type'].isin(['object',
                    'bool'])].index.values
    df_types['Count'] = df.count()
    df_types['Null Values'] = df.isnull().sum()
    df_types['Unique Values'] = df.nunique()
    df_types['Min'] = df[numerical_cols].min()
    df_types['Max'] = df[numerical_cols].max()
    df_types['Average'] = df[numerical_cols].mean()
    df_types['Median'] = df[numerical_cols].median()
    df_types['St. Dev.'] = df[numerical_cols].std()
    return df_types

# read csv files

In [ ]:
players_df=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv')
games_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/games.csv')
plays_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')
tracking_2018_df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv')


# read players_df

In [ ]:
players_df.head()

In [ ]:
explore_numerical_types(players_df)

In [ ]:
check = players_df['height'].str.split('-', expand=True)
check.columns = ['first', 'second']
check.loc[(check['second'].notnull()), 'first'] = check[check['second'].notnull()]['first'].astype(np.int16) * 12 + check[check['second'].notnull()]['second'].astype(np.int16)

using the steps from *BAEK KYUN SHIN*[ here ](https://www.kaggle.com/werooring/nfl-big-data-bowl-basic-eda-for-beginner)

in order to get 2 dicimal after the comma after converting feet to meters
> players_df['height'] = pd.options.display.float_format = "{:,.2f}".format

In [ ]:
players_df['height'] = check['first']
players_df['height'] = players_df['height'].astype(np.float32)
players_df['height'] /= 12
players_df['height']/=3.288399
players_df

In [ ]:
players_df["height"].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.histplot(players_df['height'], bins=12)
ax.set_title('Height Distribution');

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.histplot(players_df['weight'], bins=12)
ax.set_title('weight Distribution');

# convert "birthDate" field into date time type

In [ ]:
# players_df['birthDate']=players_df['birthDate'].astype("datetime64")
players_df["birthDate"]=pd.to_datetime(players_df["birthDate"],dayfirst=True)
players_df

# set a standard format for the "birthDate" field

In [ ]:
# players_df['birthDate'] = pd.to_datetime(players_df['birthDate'], format='%Y-%m-%d %H:%M:%S')
players_df['birthDate'] = pd.to_datetime(players_df['birthDate'], format='%Y-%m-%d')
players_df

# check the summary for object  or boolean types 

In [ ]:
players_df.describe(include=['object', 'bool'])

# check the value counts of the "position" field

In [ ]:
positions = players_df["Position"].value_counts().to_frame()
positions

In [ ]:
#plot
plt.figure(figsize=(10, 6))
sns.countplot(x="Position",data=players_df)

# get the *year*  *month* *age* from the **birthDate** field 

In [ ]:
# players_df["birthDate"] = pd.to_datetime(players_df["birthDate"],errors = 'coerce')
# players_df['year_month'] = players_df['birthDate'].map(lambda x: x.strftime('%Y/%m'))
players_df["year"]=players_df["birthDate"].dt.year
players_df["month"]=players_df["birthDate"].dt.month
players_df["is_leap_year"]=players_df["birthDate"].dt.is_leap_year
today = pd.to_datetime("today")
players_df["age"]=today.year - players_df["birthDate"].dt.year

players_df

In [ ]:
players_df.info()

In [ ]:
players_df["year"] =players_df["year"].fillna(0)
players_df["month"]=players_df["month"].fillna(0)
players_df["age"]= players_df["age"].fillna(0)


In [ ]:
players_df['year']=players_df['year'].astype("int")
players_df['month']=players_df['month'].astype("int")
players_df['age']=players_df['age'].astype("int")


players_df.info()

* players age are min 22 years and max 49 
* 13.9% are 26 years 

In [ ]:

age = players_df["age"].value_counts().rename_axis('age').reset_index(name='count')

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(labels=age['age'],
                             hoverinfo='label+percent',
                             values=age['count'], 
                             textposition='outside', 
                             rotation=90)])

fig.update_layout(title="Percentage of ages",
                  font=dict(family='Arial', size=12, color='#909090'),
                  legend=dict(x=0.9, y=0.5)
        )

fig.show()

In [ ]:
#plot
plt.figure(figsize=(10, 6))
sns.countplot(x="age",data=players_df)

# check the collegeName and get the value count 
looks like **alabama** college is recruiting the most players 

* plot a pie chart for the top 20

In [ ]:

col_df = players_df["collegeName"].value_counts().rename_axis('collegeName').reset_index(name='count')
col_df_top = col_df[:20].copy()
col_df_top

In [ ]:

import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(labels=col_df_top['collegeName'],
                             hoverinfo='label+percent',
                             values=col_df_top['count'], 
                             textposition='outside', 
                             rotation=90)])

fig.update_layout(title="Percentage of collegename",
                  font=dict(family='Arial', size=12, color='#909090'),
                  legend=dict(x=0.9, y=0.5)
        )

fig.show()

In [ ]:
plt.figure(figsize=(15, 12))

ax = sns.barplot(x='count', y='collegeName', data=col_df_top)
ax.set_title('Number of players for collegeName');

# the most occupied position is **CB**  and most of players are in 26 

In [ ]:
fig = go.Figure()
for name ,group in players_df.groupby("age"):
    trace = go.Histogram()
    trace.name = name
    trace.x = group["Position"]
    fig.add_trace(trace)
fig.show()

#                                                 read games_df 

In [ ]:
explore_numerical_types(games_df)

In [ ]:
games_df.head()

each season the games increased slitly  between 2018 2020

In [ ]:
games_df["season"].value_counts()

In [ ]:

sns.countplot(x = "season",data = games_df)

In [ ]:
games_df['gameDate'] = pd.to_datetime(games_df['gameDate'])


In [ ]:
games_df.info()

In [ ]:
games_df["year"]=games_df["gameDate"].dt.year
games_df["month"]=games_df["gameDate"].dt.month

* December had the biguest number of games 
* january had the lowest number of games 

In [ ]:
sns.countplot(x="month",data=games_df)

In [ ]:
games_df["homeTeamAbbr"].value_counts()

# read plays_df

In [ ]:
explore_numerical_types(plays_df)

# most of the play type was kickoff 

In [ ]:
plt.figure(figsize=(12, 6))

ax = sns.countplot(x='specialTeamsPlayType', data=plays_df)
ax.set_title('specialTeamsPlayType');

In [ ]:
kickoff =plays_df.loc[plays_df["specialTeamsPlayType"]=="Kickoff"]
kickoff

* 60% of the kickoffs are touchback
* 37.2% of teh cickoff are return

In [ ]:
kickoff = kickoff["specialTeamsResult"].value_counts().rename_axis('specialTeamsResult').reset_index(name='count')
kickoff

In [ ]:
fig = go.Figure(data=[go.Pie(labels=kickoff['specialTeamsResult'],
                             hoverinfo='label+percent',
                             values=kickoff['count'], 
                             textposition='outside', 
                             rotation=90)])

fig.update_layout(title="Percentage of kickoffs",
                  font=dict(family='Arial', size=12, color='#909090'),
                  legend=dict(x=0.9, y=0.5)
        )

fig.show()

In [ ]:
plt.figure(figsize=(12, 6))

ax = sns.countplot(x='quarter', data=plays_df)
ax.set_title('Number of plays of every quarter');

In [ ]:
plt.figure(figsize=(12, 6))

ax = sns.countplot(x='down', data=plays_df)
ax.set_title('Number of plays of every quarter');

In [ ]:
plt.figure(figsize=(14, 8))

ax = sns.countplot(x='yardsToGo', data=plays_df)
ax.set_title('Number of plays for every yards to go category');

In [ ]:
plt.figure(figsize=(12, 6))

ax = sns.histplot(plays_df['playResult'], bins = 25)
ax.set_title('playResult Distribution');

In [ ]:
 plt.figure(figsize=(12, 6))

ax = sns.histplot(plays_df['preSnapHomeScore'], bins = 12)
ax.set_title('preSnapHomeScore Distribution');

In [ ]:
 plt.figure(figsize=(12, 6))

ax = sns.histplot(plays_df['preSnapVisitorScore'], bins = 12)
ax.set_title('preSnapVisitorScore Distribution');

# read tracking_2018_df

In [ ]:
explore_numerical_types(tracking_2018_df)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
tracking_2018_df.query('gameId == 2018091001 and playId == 4033').groupby('team') \
    .plot(x='x', y='y', ax=ax, style='.')
plt.legend().remove();

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
tracking_2018_df.query('gameId == 2018091609 and position == "CB"').groupby('team') \
    .plot(x='x', y='y', ax=ax, style='.')
plt.legend().remove();